In [248]:
import warnings
warnings.filterwarnings("ignore")

import yfinance as yf
import pandas as pd

In [249]:
!pip install plotly -q

import plotly.graph_objects as go

def linear_plot(df, title):
    fig = go.Figure([go.Scatter(x=df['date'], y=df['close'], mode='lines')])
    fig.update_layout(plot_bgcolor='white', 
                      xaxis_title='Date',
                      yaxis_title='Price',
                      title=title)
    fig.show()

def candlestick_plot(df, title):
    fig = go.Figure([go.Candlestick(x=df['date'], 
                                open=df['open'],
                                high=df['high'],
                                low=df['low'],
                                close=df['close'])])
    fig.update_layout(xaxis_rangeslider_visible=False,
                      plot_bgcolor='white',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      title=title)
    fig.update_yaxes(fixedrange=False)
    fig.show()
    

In [250]:
!pip install yfinance -q

df = yf.download("AAPL", start="2024-01-01", interval="1h").drop(columns=['Adj Close'])
df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Datetime,,,,,
2024-01-02 09:30:00-05:00,187.800995,188.440002,186.009995,186.189896,21173291
2024-01-02 10:30:00-05:00,186.179993,186.904999,185.184296,186.804993,9010595
2024-01-02 11:30:00-05:00,186.800003,187.350006,186.570007,186.615005,6143754
2024-01-02 12:30:00-05:00,186.615005,187.039993,185.259995,185.294998,5381701
2024-01-02 13:30:00-05:00,185.289993,185.539993,184.229996,184.494995,7228643


In [251]:
df.shape

(1172, 5)

In [252]:
df.dtypes

Open      float64
High      float64
Low       float64
Close     float64
Volume      int64
dtype: object

In [253]:
df.isnull().sum()

Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [254]:
df.describe()

,Open,High,Low,Close,Volume
count,1172.000000,1172.000000,1172.000000,1172.000000,1.172000e+03
mean,194.362858,194.979136,193.711068,194.378666,7.028475e+06
std,20.549272,20.675305,20.387324,20.552230,5.219153e+06
min,164.369995,165.160004,164.080002,164.362396,0.000000e+00
25%,176.368755,176.968754,175.906254,176.413776,4.067693e+06
50%,188.787498,189.025002,188.295547,188.790001,5.419038e+06
75%,214.398746,215.567497,213.452503,214.365002,8.098706e+06
max,236.531998,237.229996,234.619995,235.600006,5.649606e+07


In [255]:
!pip install summarytools -q

from summarytools import dfSummary

dfSummary(df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Open[float64],Mean (sd) : 194.4 (20.5)min < med < max:164.4 < 188.8 < 236.5IQR (CV) : 38.0 (9.5),"1,093 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACxklEQVR4nO3dT4riQBTH8Vdj28Y/JIigOy8wMAuXOcQcdo7QC0/gDdwJIhHTHYwxs3JgoMfWqtSkXvh+9qW1+BF871VKU9e1AKH71vYGgEe8tL2Be4wxQxF5tVx+ruv6o8n9oD3BBtUYM5zNZj/jOJ7arD8ejwdjzC/C2g3BBlVEXuM4nqZp+pEkSfHMwizLovV6Pd3v968iQlA7IOSgiohIkiTFfD5/t1g6bHwzaA3FFFQgqFCBoEIFggoVgi+mbFVV1ReR2Bhj+xH0YQPSyaDmed4vy/LHcrns9Xq9p1pbN/Rhw/JlUDVOh87n80sURaPValUsFovDs+vpw4bnblC1T4fG47FtD1aEPmxQvnqiMh1CEB76jcp0CG2jPQUVCCpUIKhQgaBCBYIKFQgqVCCoUKGTs37YcRyXi3gcmRNUiIj7uFzE78icoOLGelwu4n9kTlDxF4dxuYjHkTnFFFQgqFCBoEIFggoVvBZTji/Yxdfrtd/wlqCUt6C6vmBXlmVUVdX3oijeRMS2CkVHeAuq6wt22+12utlsRpfLpedjf9DFex/V9gW7w+HAayz4g2IKKhBUqEBQoQKz/o5xOKoXdDuQoHaIy1G90NuBBDUwjoeX48lkMk/T9PjsUb3Q24EENSCuh5dvT8XBYPD2bEsw9HYgQQ2L0+Hl0J+KLghqgGwPL4f+VHRBewoqEFSoQFChAr9R/8HxLC1/VNEwgvoJ17O0bV8J30UE9RMuZ2m5Et4PgnqHw59VdLZN1BaKKahAUKECQYUKBBUqEFSoQNWPxvgckhBUNML3kISgohG+hyQEFY3yNSShmIIKBBUqEFSoQFChAkGFClT9Hjg0voO+VqdNBLVhLo3v0K/VaRNBbZhL47vLF0i4Iqie2DS+u3yBhKuHgpplWfTsB59Op0hEJM/zaLfbjf7neq3frXXfrusfyddvZq202aWulfEAAAAASUVORK5CYII="">",0(0.0%)
2,High[float64],Mean (sd) : 195.0 (20.7)min < med < max:165.2 < 189.0 < 237.2IQR (CV) : 38.6 (9.4),"1,076 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAC1klEQVR4nO3du47aQBSA4TNhAXORASFBR0sRKQWlHyIPm0fYgiegpaJDXGSEdx2uqYgUacNij2c9B/1fb9Za/bI0c8ZgrterAL77VvYNAI94KfsG7jHGNESklvPyw/V6fS/yflAeb0M1xjT6/f7PMAx7ea7f7XZbY8wvYn0O3oYqIrUwDHtRFL13Op00y4VxHAfT6bS3Xq9rIkKoT8DnUEVEpNPppIPB4C3HpY3CbwalYTEFFQgVKhAqVCBUqECoUOHTVb/WTffz+VwVkdAYk/cjGBh45G6oWjfdkySpHo/HH6PRqFKpVDLtwd4wMPDLZ09UlZvuh8PhJQiC5mQySYfD4Tbr9QwM/PPQhr/WTfdWq5X3vkUYGHiFxRRUIFSoQKhQgVChAqFCBUKFCoQKFbw/OI2vYzkuF3E4diZUiIj9uFzE7diZUHGTe1wu4n7sTKj4h8W4XMTh2JnFFFQgVKhAqFCBUKECoUIFQoUKhAoVnO6jWr4JGl4ul2rBtwSlnIVq+ybo8XgMzufz9zRNX0Uk7wY0noSzUG3fBF0sFr3ZbNY8nU4VF/cHXZyPUPO+CbrdbnkLFH+xmIIKHEp5MhZnSr1evBLqE7E5U+r74pVQPWN5yj5st9uDKIp2Wc+U+r54JVSP2J6yvz0V6/X6a9YFrO+LV0L1i9Upe9+fijYI1UN5T9n7/lS0wfYUVCBUqECoUIFQoQKLqf+wPKLID1UUjFA/YHtEkR+qKB6hfsDmiCI/VOEGod5h8WMVT7ufWRYWU1CBUKECoUIFQoUKhAoVWPWjMC6HJISKQrgekhAqCuF6SEKoKJSrIQmLKahAqFCBUKECoUIFQoUKhAoV2J5ywGJC4/UXlZWJUAtmM6Hx/YvKyvRQqHEcB1k/eL/fByIiSZIEy+Wy+ZXXl/m3N5tNOwiC5ng8Nt1u93eWa1erVTCfz5txHLeWy+Uhy7Uiev9nj/T1B3IqoRF8mwqdAAAAAElFTkSuQmCC"">",0(0.0%)
3,Low[float64],Mean (sd) : 193.7 (20.4)min < med < max:164.1 < 188.3 < 234.6IQR (CV) : 37.5 (9.5),"1,092 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACzElEQVR4nO3dT27aQBSA8TclRAYkW4gIdlygUhcsOUQP2yMUiRNwA3ZIFjKCBvG3m6ZqpZTgeTNlnvX99k68+GQz4yfbXa9XAVL36dEnANzj6dEncItzriMiz56HH67X62vI88HjJBuqc64zGAy+5nne9zl+s9msnXPfiLUZkg1VRJ7zPO9Pp9PXoij2dQ6sqiqbzWbDsixfnHMbz//PFTkhKYcqIiJFUeyHw+GPOsfsdrv28Xj8Mh6PW61Wq1bkb7gipyX5UH0cDoenLMu6k8lkPxqN1nWPr6oqm8/n/bIsn0WEUBPwYaiWFzS9Xq/21fgPnaAnA5WbobKgQSo+uqKqFjTcPhHKXb9RfRY0v3D7RBA8mYIJhAoTCBUmECpMIFSYQKgwgVBhAqHCBEKFCYQKEwgVJhAqTGjk4DT8KGePRSLOHxMqREQ/eywSd/6YUPHGe/ZYJP78MaHiL4rZY5GI88cspmACocIEQoUJhAoToi6mzudzW0Ry55zP4fnlcmkHPiUYFS1U7Wt1jsdjdj6fP+/3++8i4rsKRUNEC1X7Wp3lctlfLBbd0+nUinF+sCX6Pqrva3XW6zXvBMBvLKZgAqHCBB6hNoxiAirpXRZCbRDNBFTquyyE2izeE1Cp77IQamKUw8v55XJpF0WxrrvTkvouC6EmRDu8nPrtW4NQ06IaXk799q1BqAnyHV5O/fatQaj/oByo4RtVgRHqO7QDNXxkIzxCfYdmoIaPbMRBqDcovlPV2N+Kj8KzfphAqDCBWz+CiblTQqgIIvZOCaEiiNg7JYSKoGLtlLCYggmEChMIFSYQKkwgVJhAqDCBUGECocIEQoUJhAoTCBUm8Kw/AsW4W9Lvf3okQg1MM+7W5BdIaBFqYJpxtya/QEKLUCPxGXdr8gsktO4KtaqqrO4f3m63mYjIbrfLVqtV938eb/V/Wz1v7fH39PUTb

In [256]:
df.columns = df.columns.str.lower()
df.insert(0, "date", pd.to_datetime(df.index).tz_convert(tz=None))
df = df.sort_values(by='date')
df = df.drop_duplicates(subset='date').reset_index(drop=True)
df.to_csv('AAPL_1h.csv')
df

,date,open,high,low,close,volume
0,2024-01-02 14:30:00,187.800995,188.440002,186.009995,186.189896,21173291
1,2024-01-02 15:30:00,186.179993,186.904999,185.184296,186.804993,9010595
2,2024-01-02 16:30:00,186.800003,187.350006,186.570007,186.615005,6143754
3,2024-01-02 17:30:00,186.615005,187.039993,185.259995,185.294998,5381701
4,2024-01-02 18:30:00,185.289993,185.539993,184.229996,184.494995,7228643
...,...,...,...,...,...,...
1167,2024-08-30 15:30:00,228.220001,228.875000,227.550003,228.029999,3257176
1168,2024-08-30 16:30:00,228.020203,228.699997,227.479996,228.149994,2683350
1169,2024-08-30 17:30:00,228.134995,228.315002,227.660004,227.990005,2414533
1170,2024-08-30 18:30:00,228.000000,229.050003,227.949997,229.039993,3332557


In [257]:
df['date'].diff().value_counts()

date
0 days 01:00:00    1004
0 days 18:00:00     131
2 days 18:00:00      29
3 days 18:00:00       4
2 days 17:00:00       1
1 days 18:00:00       1
1 days 22:00:00       1
Name: count, dtype: int64

In [258]:
linear_plot(df, 'Apple')

In [259]:
candlestick_plot(df, 'Apple')

In [260]:
!pip install ccxt -q

import ccxt
import time

exchange = ccxt.binance()
symbol = 'BTC/USD'
timeframe = '1h'
since = exchange.parse8601('2024-01-01T00:00:00Z')
all_ohlcvs = []

while True:
    try:
        ohlcvs = exchange.fetch_ohlcv(symbol, timeframe, since)
        all_ohlcvs += ohlcvs
        if len(ohlcvs):
            print('Fetched', len(ohlcvs), symbol, timeframe, 'candles from', exchange.iso8601(ohlcvs[0][0]))
            since = ohlcvs[-1][0] + 1
            sleep_interval = exchange.rateLimit / 1000
            print('Sleep for', sleep_interval)
            time.sleep(sleep_interval)
        else:
            break
    except Exception as e:
        print(type(e).__name__, str(e))
print('Fetched', len(all_ohlcvs), symbol, timeframe, 'candles in total')

Fetched 500 BTC/USD 1h candles from 2024-01-01T00:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USD 1h candles from 2024-01-21T20:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USD 1h candles from 2024-02-11T16:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USD 1h candles from 2024-03-03T12:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USD 1h candles from 2024-03-24T08:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USD 1h candles from 2024-04-14T04:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USD 1h candles from 2024-05-05T00:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USD 1h candles from 2024-05-25T20:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USD 1h candles from 2024-06-15T16:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USD 1h candles from 2024-07-06T12:00:00.000Z
Sleep for 0.05
Fetched 500 BTC/USD 1h candles from 2024-07-27T08:00:00.000Z
Sleep for 0.05
Fetched 380 BTC/USD 1h candles from 2024-08-17T04:00:00.000Z
Sleep for 0.05
Fetched 5880 BTC/USD 1h candles in total


In [261]:
df = pd.DataFrame(all_ohlcvs)
df.columns = ['date','open','high','low','close','volume']
df = df.sort_values(by='date')
df = df.drop_duplicates(subset='date').reset_index(drop=True)
df['date'] = pd.to_datetime(df['date'], unit='ms')
df.to_csv('BTC_USD_1h.csv')
df

,date,open,high,low,close,volume
0,2024-01-01 00:00:00,42305.5,42585.1,42282.7,42494.9,1182.373496
1,2024-01-01 01:00:00,42494.7,42806.7,42450.6,42633.9,1286.649097
2,2024-01-01 02:00:00,42633.9,42659.9,42529.5,42612.3,553.320897
3,2024-01-01 03:00:00,42612.3,42617.1,42249.9,42356.2,1113.022829
4,2024-01-01 04:00:00,42356.2,42430.1,42233.8,42425.6,782.030291
...,...,...,...,...,...,...
5875,2024-09-01 19:00:00,58508.0,58740.5,58257.1,58481.7,666.598940
5876,2024-09-01 20:00:00,58481.7,58623.9,58348.2,58398.4,241.111758
5877,2024-09-01 21:00:00,58394.1,58399.6,58060.5,58313.0,143.663194
5878,2024-09-01 22:00:00,58313.1,58380.0,57165.2,57270.1,2053.329803


In [262]:
df.shape

(5880, 6)

In [263]:
df.dtypes

date      datetime64[ns]
open             float64
high             float64
low              float64
close            float64
volume           float64
dtype: object

In [264]:
df.isnull().sum()

date      0
open      0
high      0
low       0
close     0
volume    0
dtype: int64

In [265]:
df.describe()

,date,open,high,low,close,volume
count,5880,5880.000000,5880.000000,5880.000000,5880.000000,5880.000000
mean,2024-05-02 11:29:59.999999744,60008.714354,60240.190255,59759.190000,60011.226820,1831.793025
min,2024-01-01 00:00:00,38723.000000,38850.000000,38504.200000,38723.800000,66.890633
25%,2024-03-02 05:45:00,56447.075000,56753.050000,56152.925000,56447.150000,640.213941
50%,2024-05-02 11:30:00,62811.900000,63060.400000,62539.300000,62811.850000,1142.389604
75%,2024-07-02 17:15:00,66910.550000,67147.900000,66674.075000,66910.325000,2107.339675
max,2024-09-01 23:00:00,73712.400000,73915.300000,73312.000000,73712.400000,41310.160827
std,NaN,9047.137354,9078.750082,9011.913479,9044.244216,2385.062005


In [266]:
dfSummary(df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,date[datetime64[ns]],Min: 2024-01-01Max: 2024-09-01Duration: 244 days,5880 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABNCAYAAACxBha+AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACRElEQVR4nO3Yua7aUBRG4X0wIDPIFkKBjuaWkVK45CHysHmEFDwBRZpUdEjIMsL3WhjjFFGkNAyH4SeQ9bWw2QezGISr69oAhcajD4D/B7FBpnnsRudcx8zaFzzutq7rD98h9b5rvPq1uce+g7E55zrD4fBrFEUD323r9Tp1zn3zeZLqfdd49Wtzr33HPtnaURQNptPpRxzHxbnLsiwLZ7PZYLVatc3M58VX77vGq1+bu+w7+jVqZhbHcTEajd49DmpVVUVmFjnnfMai/X7fiuM4Fe0z+/2bde87ZPqzPs0+M+scuuFkbL7yPG+VZfllMpkEQRCc/a4oyzKsqupzURTfzezsJ3jpvqqqWnmev/X7/Z+NRqM8d+4RZ32WfafcPLbtdtsMw7CbJEkxHo/Tc+cWi8VgPp93d7tdINz3KUmSHz5zDzzrP7/vlJvH9kev1/P6+k3T9ODH7z33+c79PXupZ7k2t8b/bJAhNsgQG2SIDTLEBhligwyxQYbYIENskCE2yBAbZIgNMsQGGWKDDLFBhtggQ2yQITbIEBtkiA0yxAYZYoMMsUGG2CBDbJAhNsgQG2SIDTLEBhligwyxQYbYIENskCE2yBAbZIgNMsQGGWKDDLFBhtggQ2yQITbIEBtkiA0yxAYZYoMMsUGG2CBDbJAhNsgQG2SIDTLEBhligwyxQYbYIENskCE2yBAbZIgNMsQGGWKDDLFBhtggQ2yQITbINE/dIcuy0OcBN5tNaGaW53m4XC67rzb3TGdVz51q5RcE36ma7cH2JAAAAABJRU5ErkJggg=="">",0(0.0%)
2,open[float64],Mean (sd) : 60008.7 (9047.1)min < med < max:38723.0 < 62811.9 < 73712.4IQR (CV) : 10463.5 (6.6),"5,715 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACuElEQVR4nO3cP47aQBSA8TdhQQYkW/wRdFxgpRRINBwgZQ6bI6SgSM0N6JAQMsJZC2OcIomSYgXYg/G82e/Xz2JZn1iPebYpikIA131q+gCAe7w0fQBwhzGmKyIdiz9xKori7VHH8z9C9YxFbMFwOPwSRVG/6mcfDoe9MeZbHbESqkeMMd3RaPQ1DMNB2bVZlgUi8rpYLH6Mx+ND2fVxHAer1Wqw2+06IkKouKoThuFguVy+RVGUllm42WwG6/W6FwRBNplMflb8/G7FdTcRqoeiKErLxrbf72uL7BHY9UMFQoUKhAoVCBUqECpUIFSoQKhQgVChAqFCBX6ZcozlBFN4uVzajzweVxCqQ2yGSkR+D5bkef6apul3Ean6e72TCNUtlYdKRP4NlpzP51YdB9ckQnVQlaESEfcHS2ywmYIKhAoVCBUqECpUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUIExvxpYTOl7O6Fvi1AfzPbVj75O6Nsi1MezfvWjjxP6tmoN1eVXbdfNx1c/Nqm2UG0fVBOp91Xb0KXOb1SrB9XqftU2dKn9GrXqg2p/8K8QIsJ9VChBqFCBUKECoUIFQoUKhAoVCBUqECpUIFSoQKhQwdsxv488udWUPM/bIhIaY6osv3q+vQyVya3nS5KknWXZ59ls1mq1WqWHkG6dby9DFSa3nu50Or0EQdCbz+fpdDrdl1l7z/n2NVQRYXKrCf1+v+o5v3q+b4bKg2pwwdVQeVANrrj1jcqDanDCXdeoPKiGpnHDHyo4veu3uIHMRs4zzoZqcwOZjZx/nA3V5gYyGzn/OBvqX1VuILOR84/zoTaF62O3EOo7uD52D6G+g+tj9xDqFVwfu4Mb/lDhrm/UOI6Dsn/4eDwGIiJJkgTb7bb3zPVaP1vrcduuv6evX/85mtlsbUS/AAAAAElFTkSuQmCC"">",0(0.0%)
3,high[float64],Mean (sd) : 60240.2 (9078.8)min < med < max:38850.0 < 63060.4 < 73915.3IQR (CV) : 10394.8 (6.6),"5,676 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACvUlEQVR4nO3dP47aQBSA8TdhQQYUW4gIOi4QKQUlTW6Qw6ZNl4K9ADegQUgIjCCLMOAUSaQtVvyZ8aznTb5f72WKT4sZ3hhTlqUAoftQ9wKAezzVvQBUyxjTFpGW5eXHsixfqlxPVQg1IsaYdr/f/5amac/m+s1mszPG/BCRg+USvIVOqHFppWnam0wmL1mWPRTbYrH4+Pz8/HU0Gn1qNBpWoW6327Ux5ruPWAk1QlmWHQaDwa9Hrlmv1+0kSTrj8fgwHA7Xj75mnufJdDrtrVarlogQKvzqdrsPR/5Ku9LFvMKnfqhAqFCBUKECoUIFQoUKhAoVCBUqECpUIFSoQKhQga9QA+M4ppdeLpdmlesJBaEGxHVMryiK5Hw+fz4cDj9FxPb7+iARalisx/RERObzeW82m3VOp1PDx+LqRKgBshnTE/kzqudjPSHgwxRUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgWmpzxwGH6OdvDZFaFWzGX4OebBZ1eEWj3r4eeYB59deQ3V8fyPSMCP6r7F9hmlvtajnbdQXc//iPh9gjF08fkf1en8j+8nGEMX7/eotud//uKtECLCPiqUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVGB6CpU5n89NEUmNMTaXXx1AijbU/3lyqw77/b5ZFMWX0WjUsPkZ9VsDSFGGyuTW+zsej0+2P6N+zwBSlKEKk1u1cfgZ9asDSLGGKiJMbsXkZqgcVEMIrobKQTWE4tZ/VA6qIQh33aNyUA11C/rDlMMGMvfHkQk2VJcNZO6P4xNsqC4byNwfxyfYUP+x2UDm/jg+TE9BBUKFCsG/9deFHYewEOob2HEID6G+gR2H8BDqFew4hOOuUPM8Tx79w7vdLhER2e/3yXK57Lzn9VpfW+u6Xa+/p6/f5smjLNoaTLYAAAAASUVORK5CYII="">",0(0.0%)
4,low[float64],Mean (sd) : 59759.2 (9011.9)min < med < max:38504.2 < 62539.3 < 73312.0IQR (CV) : 10521.1 (6.6),"5,671 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgA

In [267]:
linear_plot(df, 'BTC/USD')

In [268]:
candlestick_plot(df, 'BTC/USD')

In [269]:
df['date'].diff().value_counts()

date
0 days 01:00:00    5879
Name: count, dtype: int64